In [7]:
#Import libraries
from urllib.request import urlopen
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [45]:
#Import list of job openings
#Depending on the page being scraped, comment out the page number
jobpage = "https://euraxess.ec.europa.eu/jobs/search?keywords=pharmaceutical" + "&page=1"
page = urlopen(jobpage)
soup = BeautifulSoup(page, "lxml")

In [43]:
#Parse the webpage into various fields
def addtable(isoup, df, link):
    jobtitle = [x.text.strip() for x in isoup.find_all("h1", class_="head-title")]
    body = [x.text.strip() for x in isoup.find_all("div", class_="longtext field-body")]
    organization = [x.text for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-company-institute inline")]
    field = [[y.strip() for y in x.text.strip().split(" \n\n")] for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-research-field inline")]
    profile = [[y.strip() for y in x.text.strip().split(" \n\n")] for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-research-profile inline")]
    location = [x.text.strip().split(" \n\n") for x in isoup.find_all("div", class_="col-xs-12 col-sm-7 value field-country inline")]
    city = [x.text.strip() for x in isoup.find_all("div", class_="value field-city")]
    addinfo = [x.text.strip() for x in isoup.find_all("div", class_="col-xs-12 field-additional-information")]
    reqt = [x.text.strip() for x in isoup.find_all("div", {"id":"bootstrap-fieldgroup-accordion-item--requirements-1"})]
    timex = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    newrow = {"Jobtitle":jobtitle, 
              "Organization":organization, 
              "Field":field,
              "Profile":profile, 
              "Location":location, 
              "City":city, 
              "Requirements":reqt, 
              "Description":body, 
              "Misc":addinfo,
             "Link": link,
             "Time": timex}
    return newrow

In [48]:
#Parse each job opening result into a pandas dataframe
for x in soup.find_all("div", class_="col-sm-12 col-md-6"):
    jobscsv = pd.read_pickle("jobs.pkl")
    print(x.a["href"])
    if x.a["href"] not in jobscsv["Link"].unique():
        indpage=urlopen("https://euraxess.ec.europa.eu" + str(x.a["href"]))
        isoupx = BeautifulSoup(indpage, "lxml")
        newrow = addtable(isoupx, jobscsv, str(x.a["href"]))
        jobscsv =jobscsv.append(newrow, ignore_index=True)
        jobscsv.to_pickle("jobs.pkl")
        jobscsv.to_csv("jobs.csv")
        for i in range(60*20):
            time.sleep(1)

/jobs/178426
/jobs/178432


KeyboardInterrupt: 

In [100]:
import six
from ast import literal_eval

In [135]:
#Sample parsing
joblist = pd.read_pickle("jobs.pkl")
joblist["Link"] = joblist["Link"].str.extract("([1-9]+)")
joblist["Organization"] = joblist["Organization"].apply(lambda x: literal_eval(str(x))[0].strip())
joblist["Jobtitle"] = joblist["Jobtitle"].apply(lambda x: literal_eval(str(x))[0].strip())
joblist["Field"] = joblist["Field"].apply(lambda x: literal_eval(str(x))[0])
joblist["Profile"] = joblist["Profile"].apply(lambda x: literal_eval(str(x))[0])
joblist["Location"] = joblist["Location"].apply(lambda x: literal_eval(str(x))[0][0])
joblist["City"] = joblist["City"].apply(lambda x: literal_eval(str(x))[0])
joblist["Requirements"] = joblist["Requirements"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist["Description"] = joblist["Description"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist["Misc"] = joblist["Misc"].apply(lambda x: literal_eval(str(x))[0] if len(literal_eval(str(x)))>0 else x)
joblist.iloc[10]

C:\Users\ara730\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  app.launch_new_instance()


Link                                                       178426
Jobtitle                    Neuroinflammation / Neuroimmunologist
Organization                                Astex Pharmaceuticals
Field             [Biological sciences, Chemistry, Neurosciences]
Profile         [First Stage Researcher (R1), Recognised Resea...
Location                                           United Kingdom
City                                                    Cambridge
Requirements    Offer RequirementsSkills/QualificationsPostgra...
Description     Astex Pharmaceuticals is a world leader in inn...
Misc            Additional commentsContract\nWeb site for addi...
Time                                          2017-02-27 11:34:29
Name: 10, dtype: object